In [4]:
#import basic Libraries
import pandas as pd
import numpy as np
from tqdm import tqdm

#Visualizaiton imports
import seaborn as sns
import matplotlib.pyplot as plt

#Importing Sklearn
from sklearn.preprocessing import StandardScaler, OneHotEncoder

#Import Market Basket Models
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

%matplotlib inline

In [5]:
#import of all files from the data folder.
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')
products = pd.read_csv('../data/products.csv')
order_products__prior = pd.read_csv('../data/order_products__prior.csv')
order_products__train = pd.read_csv('../data/order_products__train.csv')
orders = pd.read_csv('../data/orders.csv')
merged_products_aisles_departments = pd.read_csv('../data/merged_data.csv')
merged_products_aisles_departments = merged_products_aisles_departments.drop('Unnamed: 0', axis=1)

In [ ]:
orders

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


# Preprocessing

In [ ]:
opt = order_products__train.merge(merged_products_aisles_departments, how='left', on='product_id')

In [ ]:
opt.head()

In [ ]:
#converting the id to string to preserve order.
opt['aisle_id']= opt['aisle_id'].astype(str)

In [ ]:
#instanciate Oone hot encoder ot break out the data my aisle.
ohe = OneHotEncoder(handle_unknown='ignore')

In [ ]:
#test to  see # of aisles
print('# of unique aisles =',len(opt['aisle_id'].unique()))
aisle = ohe.fit_transform(opt['aisle_id'].astype(str).values.reshape(-1,1))

In [ ]:
#one hot encode the aisles and create a dense matrix.
a_ohe = pd.DataFrame(aisle.toarray())
aisle = a_ohe.to_dense()

In [ ]:
aisle.head()

In [ ]:
#add order_id to the one hot encoded matrix
basket = opt[['order_id']].join(aisle)

In [ ]:
basket.head()

In [ ]:
range(len(opt['aisle_id'].unique()))

In [ ]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [ ]:
basket.head()

In [ ]:
#covert any values to be a binary. 
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [ ]:
basket_sets.head()

In [ ]:
aisle_dict = dict(zip(opt['aisle_id'].unique().astype('Int64'),opt['aisle'].unique()))

In [ ]:
basket_sets.columns = list(range(1,len(aisle_dict)+1))
basket_sets.rename(columns=aisle_dict,inplace=True)

In [ ]:
basket_sets.head()

In [ ]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

Apriori is an algorithm for extracting frequent itemsets with applications in association rule learning. The apriori algorithm has been designed to operate on databases containing transactions, such as purchases by customers of a store. An itemset is considered as "frequent" if it meets a user-specified support threshold. For instance, if the support threshold is set to 0.5, a frequent itemset is defined as a set of items that occur together in at least 50% of all transactions in the database.

# Market Basket Results

In [ ]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
rules.head()

Support is the relatvie frequency that the rules show up.
Confidence is a measure of the reliability of the rule.
Lift is the ratio of the observed support to that expected if the two rules were independet.

In [ ]:
#sorts to see the lift greater than or equal to 1
rules[rules['lift'] >= 1.5].head()

# Market Basket for Products

In [ ]:
reorders = opt[opt['reordered'] == 1]
#opt[opt['reordered'] == 1]

In [ ]:
reorders['product_id'] = reorders['product_id'].astype('int64')

In [ ]:
# get list of hi volume products (products that occurr more than 1 time)
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [ ]:
# mask the reorders dataframe to only incluse dem hi
reorders = reorders[reorders['product_id'].isin(hivol)]

In [ ]:
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)

In [ ]:
hidem_ord = reorders[reorders['hi_dem'] == True]

In [ ]:
hidem_ord = hidem_ord.reset_index()

In [ ]:
print('# of unique products =',len(hidem_ord['product_id'].unique()))
#tranposes the data to ensure that it is correctly fitted.
product = ohe.fit_transform(hidem_ord['product_id'].values.reshape(-1,1))

In [ ]:
p_ohe = pd.DataFrame(product.toarray())
products = p_ohe.to_dense()

In [ ]:
basket = hidem_ord[['order_id']].join(products)

In [ ]:
#return the total items in the basket from the aisles
basket = basket.groupby('order_id').sum()

In [ ]:
#covert any values to be a binary.
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)

In [ ]:
pro_dict = dict(zip(hidem_ord['product_id'].unique().astype('Int64'),hidem_ord['product_name'].unique()))

In [ ]:
pro_dict = dict(zip(list(range(0,len(hidem_ord['product_id'].unique()))),hidem_ord['product_name'].unique()))

In [ ]:
#rename the columns for easier analysis in rules. 
basket_sets.columns = list(range(0,len(pro_dict)))
basket_sets.rename(columns=pro_dict,inplace=True)

In [ ]:
basket_sets.sum().head()

In [ ]:
#creates conditions for the items in the basket.
frequent_itemsets = apriori(basket_sets, min_support=0.0005, use_colnames=True)

In [ ]:
#creates the rules to filter.
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)

In [ ]:
rules[(rules['lift'] >= 100) & (rules['confidence']>.5)]

In [ ]:
rules